### Main Loop for the analysis of a list of votables  ...

Features are:
1. Ability to stop and resume loop
2. Ability to plot the results
3. Plug different kinds of analysis.
4. Log results in a file and/or sqlitedb
5. Should be runnable in the downloaded jl file.
6. ETA

In [91]:
using DataFrames
using CSV

push!(LOAD_PATH,"/home/stephane/Science/cluster/GAIA/master/src")
using GaiaClustering


## directory
rootdir = "/home/stephane/Science/cluster/GAIA"
wdir    = "/home/stephane/Science/cluster/GAIA/products"
votdir  = "/home/stephane/Science/cluster/GAIA/products/votable"

cd(wdir)

In [92]:
## load a liist of votable and update the file if done
## add results
## 

function readlist_votable(filelist::String)
    vot = CSV.read(filelist)
    return(vot)
end

function init_result(votname,c1 , c2 , c3 , fileres::String)
    res = DataFrame(votname=votname,c1 = c1 , c2 = c2 , c3=c3)
    CSV.write(fileres,res,delim=';')
    return(res)
end

function add_result!(res, votname,c1, c2, c3, fileres::String)
    newrow = DataFrame(votname=votname,c1 = c1 , c2 = c2 , c3 = c3)
    println("### add results .....")
    append!(res,newrow)
    CSV.write(fileres,res,delim=';')
end

add_result! (generic function with 1 method)

In [93]:
function getdata(filevot)
    voname = filevot

    data       = read_votable(voname)
    df         = filter_data(data)
    dfcart     = add_cartesian(df)
    blck       = [[1,2,3],[4,5], [6,7,8]]
    wghtblck   = [4.0,5.0,1.0]
    norm       = "identity"

    dfcartnorm , scale8 = normalization_PerBlock(dfcart, blck, wghtblck , norm, false) 
    return(dfcartnorm , dfcart)
end

getdata (generic function with 1 method)

In [94]:
## Main loop
##

macro main(filelist,fileres)
    let
        println("## Starting main loop ..")
        vot = readlist_votable(filelist)
        println("## $filelist read")
    
        ## main loop
        res = [0]
        s=size(vot)
        
        ## try to read the result file.
        initfound = true
        try
          res = CSV.read(fileres, delim=";")
        catch
            print("## No $fileres, it will be created...")
            initfound = false
        end
        
        for i in 1:s[1]
            votname = vot[:votname][i]
            votfound = false   ## variable to check if vot already analyzed...
            if initfound
                if votname in res[:votname] 
                    println("## $votname already done.")
                    votfound = true
                end
            end
            
            if !votfound
                println("## Starting with $votname")
                # dfcartnorm , dfcart = getdata(votdir*"/"*votname)
                
                if !initfound
                    res = init_result(votname,2,4,2.1,fileres)
                    initfound = true
                else
                    add_result!(res, votname, 2,7,103.22,fileres)  
                end
                
            end
        end
    end
    print("## Main loop done.")
end

@main("votlist.csv", "votResults.csv")

## Starting main loop ..
## votlist.csv read
## toto already done.
## MWSC 4674-1.1deg.vot already done.
## MWSC 5011-0.8deg.vot already done.
## Main loop done.